## Convert .bin drawing to .npy drawing

### Import

In [22]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import re

### Load data

In [23]:
object_file = open("object.txt", "r")
objects = object_file.readlines()
object_file.close()
N_CLASSES = len(objects)
CLASSES = {}
for idx, obj in enumerate(objects):
    CLASSES[idx] = obj.replace('\n', '')
print(CLASSES)



{0: 'triangle', 1: 'circle', 2: 'square', 3: 'apple', 4: 'banana', 5: 'diamond'}


In [24]:
def extract_label(file_path):
    """
    Extract the variable from a string in the format npy_data\{variable}.npy.

    Parameters:
    - file_path: String containing the path.

    Returns:
    - str: Extracted variable.
    """
    match = re.search(r'npy_data\\(.*?)\.npy', file_path)
    if match:
        variable = match.group(1)
        return variable
    else:
        return None
    
def get_key_by_value(dictionary, target_value):
    """
    Get the key associated with a specific value in a dictionary.

    Parameters:
    - dictionary: The input dictionary.
    - target_value: The value to search for.

    Returns:
    - key: The key associated with the target value, or None if not found.
    """
    for key, value in dictionary.items():
        if value == target_value:
            return key
    return None

In [25]:
# Get random 1000 drawings from each object
def load_drawings(root, reshaped=False):
    all_files = glob.glob(os.path.join(root, '*.npy'))
    
    data = []
    label_data = []
    for index, file in enumerate(all_files):
        extracted_label = extract_label(file)
        np_arrays = np.load(file)
        label_arrays = []
        if reshaped:
            new_arrays = []
            new_label_arrays = []
            for idx in range(len(np_arrays)):
                label_index = get_key_by_value(CLASSES, extracted_label)
                if label_index == None:
                    continue
                np_arr = np.reshape(np_arrays[idx] ,(28,28,1))
                new_arrays.append(np_arr)
                new_label_arrays.append(label_index)
            np_arrays = new_arrays
            label_arrays = new_label_arrays
        data.append(np_arrays)
        label_data.append(label_arrays)
    return data, label_data

In [26]:
data, label_data = load_drawings('npy_data', reshaped=True)

In [27]:
print(len(label_data))

6


In [28]:
print(len(data))

6


In [29]:
def visualize(nparr):
    img = Image.fromarray(nparr.reshape(28,28))
    img.show(title="visualize array")

In [30]:
def set_lim(np_arrays, label_data_arrays, lim):
    lim_arr = []
    lim_labels = []
    for arr_index in range(len(np_arrays)):
        i = 0
        data_array = np_arrays[arr_index]
        label_array = label_data_arrays[arr_index]
        for index in range(len(data_array)):
            if i == lim:
                break
            lim_arr.append(data_array[index])
            lim_labels.append(label_array[index])
            i += 1
    return lim_arr, lim_labels

In [31]:
data, labels = set_lim(data, label_data, 5000)

In [32]:
print(len(data))
print(len(labels))
print(labels[2000:2010])

30000
30000
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


In [33]:
def normalize(nparr):
    return np.interp(nparr, [0, 255], [-1,1])

In [34]:
def denormalize(nparr):
    return np.interp(nparr, [-1, 1], [0, 255])

In [35]:
data = normalize(data)

print(len(labels))

30000


### Training

In [36]:
from sklearn.model_selection import train_test_split as tts
from keras.utils import np_utils

In [37]:
x_train, x_test, y_train, y_test = tts(data, labels, test_size=0.2)

In [38]:
Y_train = np_utils.to_categorical(y_train, N_CLASSES)
Y_test = np_utils.to_categorical(y_test, N_CLASSES)

In [39]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten


def conv(classes, input_shape):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(classes, activation='softmax'))
    return model


In [40]:
model = conv(classes=N_CLASSES, input_shape=(28,28,1))

In [41]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [42]:
EPOCHS = 20
model.fit(np.array(x_train), np.array(Y_train), batch_size=32, epochs=EPOCHS, verbose=1)

Epoch 1/20
750/750 [==============================] - 35s 45ms/step - loss: 0.2723 - accuracy: 0.9163
Epoch 2/20
750/750 [==============================] - 30s 39ms/step - loss: 0.1525 - accuracy: 0.9535
Epoch 3/20
750/750 [==============================] - 31s 42ms/step - loss: 0.1350 - accuracy: 0.9572
Epoch 4/20
750/750 [==============================] - 34s 45ms/step - loss: 0.1180 - accuracy: 0.9639
Epoch 5/20
750/750 [==============================] - 33s 43ms/step - loss: 0.1085 - accuracy: 0.9659
Epoch 6/20
750/750 [==============================] - 30s 40ms/step - loss: 0.0956 - accuracy: 0.9694
Epoch 7/20
750/750 [==============================] - 32s 42ms/step - loss: 0.0898 - accuracy: 0.9711
Epoch 8/20
750/750 [==============================] - 35s 46ms/step - loss: 0.0862 - accuracy: 0.9715
Epoch 9/20
750/750 [==============================] - 32s 42ms/step - loss: 0.0763 - accuracy: 0.9748
Epoch 10/20
750/750 [==============================] - 27s 35ms/step - loss: 0.071

In [43]:
pred = model.predict(np.array(x_test))

188/188 [==============================] - 1s 3ms/step


In [44]:
score = 0

for i in range(len(pred)):
    if np.argmax(pred[i]) == y_test[i]:
        score+=1

acc = ((score+0.0)/len(pred)*100)
print("Accuracy: {acc}".format(acc = ((score+0.0)/len(pred)*100)))

Accuracy: 96.71666666666667


In [45]:
from datetime import datetime
# Get the current date and time
current_time = datetime.now()

# Format the date and time as a string (e.g., "2022-01-01_12-30-45")
formatted_time = current_time.strftime("%Y-%m-%d_%H-%M-%S")

# Combine the formatted time with a base file name
base_file_name = "./model/doodle"

file_name_with_timestamp = f"{base_file_name}_{formatted_time}.h5"

model.save(file_name_with_timestamp)

In [46]:
from keras.models import load_model

model = load_model("./model/doodle.h5")


In [47]:
def load_numpy_array(file_path):
    """
    Load a NumPy array from a file.

    Parameters:
    - file_path: File path to load the array from.

    Returns:
    - numpy array: Loaded NumPy array.
    """
    loaded_array = np.load(file_path)
    return loaded_array

# Example Usage:
file_path = "./user-data.npy"
loaded_array = load_numpy_array(file_path)
print("Loaded NumPy array:")
visualize(denormalize(loaded_array))
print(model.predict(loaded_array))
pred = CLASSES[np.argmax(model.predict(loaded_array))]
print("Predicted:", pred)

Loaded NumPy array:
1/1 [==============================] - 0s 99ms/step
[[8.60616505e-01 1.39372781e-01 1.06889265e-05]]
1/1 [==============================] - 0s 23ms/step
Predicted: triangle


In [48]:
from random import randint
def visualize_and_predict():
    "selects a random test case and shows the object, the prediction and the expected result"
    n = randint(0, len(x_test))
    visualize(denormalize(np.reshape(x_test[n], (28, 28))))
    pred = CLASSES[np.argmax(model.predict(np.array([x_test[n]])))]
    actual = CLASSES[y_test[n]]
    print("Actual:", actual)
    print("Predicted:", pred)


In [49]:
visualize_and_predict()

1/1 [==============================] - 0s 21ms/step
Actual: triangle
Predicted: triangle
